<a href="https://colab.research.google.com/github/KairaNithin/XML-Experiments/blob/main/TPU_single_core_success.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers
!pip install transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 81kB 4.7MB/s 
     |████████████████████████████████| 2.3MB 28.5MB/s 
     |████████████████████████████████| 1.2MB 50.7MB/s 
     |████████████████████████████████| 901kB 53.7MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=ffab6a46a0e849d912320e10041fb49ade495c1dc0145f094b30fe4a5b8d9007
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [2]:
import torch
if torch.cuda.is_available():
  print("There are %d Gpu's available"%torch.cuda.device_count())
  print("using the {} GPU".format(torch.cuda.get_device_name(0)))
else :
  print("No Gpu's available, using the cpu instead")

No Gpu's available, using the cpu instead


In [3]:
import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
# Check where we need this
# from nltk.corpus import stopwordsm
from nltk.tokenize import sent_tokenize

import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelWithLMHead


import torch.nn as nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import gensim.models as gsm

from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_dataloader, valid_dataloader, dataloader, dataloaders = torch.load('drive/MyDrive/train_dataloader.pth'), torch.load('drive/MyDrive/valid_dataloader.pth'),torch.load('drive/MyDrive/dataloader.pth'),torch.load('drive/MyDrive/dataloaders.pth')

In [6]:
class ClassificationHead(nn.Module):
  """ Classification head for the Roberta Model """ 
  def __init__(self, numberOfClasses, hidden_size_bert, hidden_size_post_feats, dropout_val = 0.2):
    super().__init__()
    self.denseInit = nn.Linear(hidden_size_post_feats, hidden_size_bert)
    self.dense = nn.Linear(hidden_size_bert, hidden_size_bert)
    self.dropout = nn.Dropout(dropout_val)
    self.output = nn.Linear(hidden_size_bert, numberOfClasses)
  def forward(self, x):
    # print(x.shape)
    x = self.dropout(x)
    x = self.denseInit(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.dense(x)
    x  = torch.tanh(x)
    x = self.dropout(x)
    x  = self.output(x)
    return x

class TextClassification(nn.Module):
  """ Classifier with feature injection """
  def __init__(self, numberOfClasses,dropout_val = 0.1, batch_size = 16):
     super(TextClassification, self).__init__()
     self.bert = XLMRobertaModel.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens")
     self.classifier = ClassificationHead(numberOfClasses, self.bert.config.hidden_size, (self.bert.config.hidden_size * 2 + 300) , dropout_val)
  def forward(self, input_seq, attention_mask, emoji, hashTag):
    bert_pooled_output = self.bert(input_seq, attention_mask=attention_mask)[0]
    bert_pooled_output = bert_pooled_output[:, 0, :]
    bert_pooled_out_feat = torch.cat([bert_pooled_output, emoji, hashTag], axis = 1)
    # print("Shape",bert_pooled_out_feat.shape)
    output = self.classifier(bert_pooled_out_feat)
    return output

In [7]:
model_name = 'adaptive'
model_loc = '/content/drive/My Drive/hasoc_saved/'

In [8]:
def modelEvaluate(model, valid_dataloader = valid_dataloader, task = 1):
  gc.collect()
  if task == 1:
    taskIndex = 6
  elif task == 2:
    taskIndex = 7
  model.eval()
  predictions, true_labels = [], []
  logits = []
  # Predict 
  for batch in valid_dataloader:
    # Add batch to GPU
    b_input_ids = batch[1]
    b_input_mask = batch[2]
    b_labels = batch[taskIndex]
    b_emoji = batch[5]
    b_hashtag = batch[4]
    with torch.no_grad():
      pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
    logits.append(pred.detach().cpu().numpy())
    label_ids = b_labels.to('cpu').numpy()
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    flat_true_labels = np.concatenate(true_labels, axis = 0)
    predictions = []
    for i in logits:
      for j in i:
        predictions.append(j)
    flat_predictions = [np.argmax(i) for i in predictions]
    assert(len(flat_predictions) == len(flat_true_labels))
    return flat_predictions, flat_true_labels

In [9]:
path =  model_name + ".pt"
scale = 1

In [10]:
def make_optim(model, rate = 2e-5):
  return AdamW(model.parameters(),
                lr = rate, # default = 5e-5, using 2e-5
                eps = 1e-8) # default = 1e-8

def train_model(train_dataloader, valid_dataloader, numberOfEpochs = 10, task = 1):
  """ Train Loop for the model """
  scale = 1
  if task == 2:
    classNum = 4
    taskIndex = 7
  elif task == 1:
    classNum = 2
    taskIndex = 6
  else:
    raise NameError("Task not defined")
  total_steps = len(train_dataloader)
  print("Start")

  model = TextClassification(classNum) # task 1 
  
  if torch.device == "gpu":
    model.cuda()
  device = torch.device("cuda")
  
  loss_function = nn.CrossEntropyLoss().to(device)
  epoch_loss = 0
  batch_accuracy_scores = []
  global_pred = []
  global_label = []

  present_rate = 2e-5
  old_best = -1
  epoch = 0

  while(1):
    # when the learn rate falls below a lower threshold, you stop your training
    # until that moment, march on
    epoch += 1
    print("\nEpoch:", epoch)
    print("Present Rate: " + str(present_rate))
    optimizer = make_optim(model, present_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
    gc.collect()
    model.train()
    epoch_loss = 0
    batch_accuracy_scores = []
    train_data_count = float(len(train_dataloader))

    # to check if performance with default weights
    predictions, true_labels = modelEvaluate(model, valid_dataloader, task)
    score_now = f1_score(true_labels, predictions, average = 'macro')
    print("Validation Macro: " + str(score_now))

    if (score_now > old_best):
      print("Continuing on track")
      old_best = score_now

      # delete previous best 
      delete_filename = path
      open(delete_filename, 'w').close() # overwrite and make the file blank instead
      os.remove(delete_filename) # delete the blank file from google drive will move the file to bin instead
      torch.save(model.state_dict(), path)

    else:
      print("Backtrack")
      model.load_state_dict(torch.load(path))
      present_rate /= (4 * scale)
      scale *= 4
      if present_rate < 1e-8:
        break

    # For quick eval
    cnt = 0
    # for i, batch in tqdm(enumerate(train_dataloader)):
    for i, batch in enumerate(train_dataloader):
        print("Iter: " + str(cnt + 1))
        # COMMENT OUT THE NEXT 2 LINES IN ACTUAL TRAINING
        # if cnt == 4:
        #   break
        cnt += 1
        b_input_ids = batch[1]
        b_input_mask = batch[2]
        b_labels = batch[taskIndex]
        b_emoji = batch[5]
        b_hashtag = batch[4]
        pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
        loss = loss_function(pred.view(-1, classNum), b_labels.view(-1))
        with torch.no_grad():
          epoch_loss += (loss.item() * len(b_labels))
          global_pred.append(pred)
          global_label.append(b_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

  return model

In [ ]:
gc.collect()
model = train_model(train_dataloader, valid_dataloader, 2, task = 1)

Start




Epoch: 1
Present Rate: 2e-05
Validation Macro: 0.4482758620689655
Continuing on track
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7
Iter: 8
Iter: 9
Iter: 10
Iter: 11
Iter: 12
Iter: 13
Iter: 14
Iter: 15
Iter: 16
Iter: 17
Iter: 18
Iter: 19
Iter: 20
Iter: 21
Iter: 22
Iter: 23
Iter: 24
Iter: 25
Iter: 26
Iter: 27
Iter: 28
Iter: 29
Iter: 30
Iter: 31
Iter: 32
Iter: 33
Iter: 34
Iter: 35
Iter: 36
Iter: 37
Iter: 38
Iter: 39
Iter: 40
Iter: 41
Iter: 42
Iter: 43
Iter: 44
Iter: 45
Iter: 46
Iter: 47
Iter: 48
Iter: 49
Iter: 50
Iter: 51
Iter: 52
Iter: 53
Iter: 54
Iter: 55
Iter: 56
Iter: 57
Iter: 58
Iter: 59
Iter: 60
Iter: 61
Iter: 62
Iter: 63
Iter: 64
Iter: 65
Iter: 66
Iter: 67
Iter: 68
Iter: 69
Iter: 70
Iter: 71
Iter: 72
Iter: 73
Iter: 74
Iter: 75
Iter: 76
Iter: 77
Iter: 78
Iter: 79
Iter: 80
Iter: 81
Iter: 82
Iter: 83
Iter: 84
Iter: 85
Iter: 86
Iter: 87
Iter: 88
Iter: 89
Iter: 90
Iter: 91
Iter: 92
Iter: 93
Iter: 94
Iter: 95
Iter: 96
Iter: 97
Iter: 98
Iter: 99
Iter: 100
Iter: 101
Iter: 102


In [ ]:
 torch.save(model,"drive/MyDrive/simple_model.pt")

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install pytorch-lightning

     |████████████████████████████████| 144.6MB 57kB/s 
     |████████████████████████████████| 61kB 3.3MB/s 
ERROR: earthengine-api 0.1.264 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
     |████████████████████████████████| 808kB 5.3MB/s 
     |████████████████████████████████| 829kB 12.5MB/s 
     |████████████████████████████████| 276kB 22.7MB/s 
     |████████████████████████████████| 645kB 24.7MB/s 
     |████████████████████████████████| 112kB 36.8MB/s 
     |████████████████████████████████| 1.3MB 29.6MB/s 
     |████████████████████████████████| 296kB 41.5MB/s 
     |████████████████████████████████| 143kB 37.2MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=1867014f5d868493d3a888449

In [2]:

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

class LitAutoEncoder(pl.LightningModule):
	def __init__(self):
		super().__init__()
		self.encoder = nn.Sequential(
      nn.Linear(28 * 28, 64),
      nn.ReLU(),
      nn.Linear(64, 3))
		self.decoder = nn.Sequential(
      nn.Linear(3, 64),
      nn.ReLU(),
      nn.Linear(64, 28 * 28))

	def forward(self, x):
		embedding = self.encoder(x)
		return embedding

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
		return optimizer

	def training_step(self, train_batch, batch_idx):
		x, y = train_batch
		x = x.view(x.size(0), -1)
		z = self.encoder(x)    
		x_hat = self.decoder(z)
		loss = F.mse_loss(x_hat, x)
		self.log('train_loss', loss)
		return loss

	def validation_step(self, val_batch, batch_idx):
		x, y = val_batch
		x = x.view(x.size(0), -1)
		z = self.encoder(x)
		x_hat = self.decoder(z)
		loss = F.mse_loss(x_hat, x)
		self.log('val_loss', loss)

# data
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

train_loader = DataLoader(mnist_train, batch_size=32)
val_loader = DataLoader(mnist_val, batch_size=32)

# model
model = LitAutoEncoder()

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw




Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw




Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
for batch in train_loader:
  break

In [3]:
import pytorch_lightning as pl

trainer = pl.Trainer(tpu_cores=8)
# trainer.fit(my_model)


# training
# trainer = pl.Trainer(gpus=4, num_nodes=8, precision=16, limit_train_batches=0.5)
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: cleaning up ddp environment...
  warnings.warn(*args, **kwargs)


KeyboardInterrupt: ignored

In [1]:
!pip install sentence_transformers
!pip install transformers
from sentence_transformers import SentenceTransformer

In [2]:
import random
import pickle
import re
import time
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
# Check where we need this
# from nltk.corpus import stopwordsm
from nltk.tokenize import sent_tokenize

import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelWithLMHead


import torch.nn as nn
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import gensim.models as gsm

from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


from tqdm import tqdm 
import gc
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_dataloader, valid_dataloader, dataloader, dataloaders = torch.load('drive/MyDrive/train_dataloader.pth'), torch.load('drive/MyDrive/valid_dataloader.pth'),torch.load('drive/MyDrive/dataloader.pth'),torch.load('drive/MyDrive/dataloaders.pth')

In [9]:
a = None
b = None
c = None
d = None
import pytorch_lightning as pl

class TextClassification(pl.LightningModule):
  """ Classifier with feature injection """
  def __init__(self, numberOfClasses,dropout_val = 0.1, batch_size = 16):
    super(TextClassification, self).__init__()
    self.bert = XLMRobertaModel.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens")
    # self.classifier = ClassificationHead(numberOfClasses, self.bert.config.hidden_size, (self.bert.config.hidden_size * 2 + 300) , dropout_val)
  def forward(self, input_seq, attention_mask, emoji, hashTag):
    bert_pooled_output = self.bert(input_seq, attention_mask=attention_mask)[0]
    bert_pooled_output = bert_pooled_output[:, 0, :]
    bert_pooled_out_feat = torch.cat([bert_pooled_output, emoji, hashTag], axis = 1)
    # print("Shape",bert_pooled_out_feat.shape)
    # output = self.classifier(bert_pooled_out_feat)
    return 0.0

  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5, eps=1e-8)
    return optimizer

  def training_step(self, train_batch, batch_idx):

    b_input_ids = train_batch[1]
    b_input_mask = train_batch[2]
    b_labels = train_batch[6]
    b_emoji = train_batch[5]
    b_hashtag = train_batch[4]
    print("b_hashtag")
    pred = self.forward(b_input_ids,b_input_mask,b_emoji.float(), b_hashtag.float())
    print("executed")
    loss = 0.02
    # x, y = train_batch
    # x = x.view(x.size(0), -1)
    # z = self.encoder(x)    
    # x_hat = self.decoder(z)
    # loss = F.mse_loss(x_hat, x)
    self.log('train_loss', loss)
    return loss

In [10]:
model = TextClassification(2)

In [11]:
import pytorch_lightning as pl

trainer = pl.Trainer(tpu_cores=8)
# trainer.fit(my_model)


trainer.fit(model, train_dataloader)

INFO:pytorch_lightning.utilities.distributed:GPU available: False, used: False
INFO:pytorch_lightning.utilities.distributed:TPU available: True, using: 8 TPU cores
INFO:pytorch_lightning.core.lightning:
  | Name | Type            | Params
-----------------------------------------
0 | bert | XLMRobertaModel | 278 M 
-----------------------------------------
278 M     Trainable params
0         Non-trainable params
278 M     Total params
1,112.175 Total estimated model params size (MB)


b_hashtag
b_hashtag
executed
executed


Exception in device=TPU:6: unsupported operand type(s) for /: 'NoneType' and 'int'
Exception in device=TPU:7: unsupported operand type(s) for /: 'NoneType' and 'int'
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 329, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 329, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 323, in _start_fn
    fn(gindex, *args)
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/plugins/training_type/tpu_spawn.py", line 163, in new_process
    results = trainer.run_stage()
  File "/usr/local/lib/python3.7/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 323, in _start_fn
    fn(gindex, *args)
  File 

ProcessExitedException: ignored

In [3]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

class ClassificationHead(nn.Module):
  """ Classification head for the Roberta Model """ 
  def __init__(self, numberOfClasses, hidden_size_bert, hidden_size_post_feats, dropout_val = 0.2):
    super().__init__()
    self.denseInit = nn.Linear(hidden_size_post_feats, hidden_size_bert)
    self.dense = nn.Linear(hidden_size_bert, hidden_size_bert)
    self.dropout = nn.Dropout(dropout_val)
    self.output = nn.Linear(hidden_size_bert, numberOfClasses)
  def forward(self, x):
    # print(x.shape)
    x = self.dropout(x)
    x = self.denseInit(x)
    x = torch.tanh(x)
    x = self.dropout(x)
    x = self.dense(x)
    x  = torch.tanh(x)
    x = self.dropout(x)
    x  = self.output(x)
    return x

class TextClassification(nn.Module):
  """ Classifier with feature injection """
  def __init__(self, numberOfClasses,dropout_val = 0.1, batch_size = 16):
     super(TextClassification, self).__init__()
     self.bert = XLMRobertaModel.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens")
     self.classifier = ClassificationHead(numberOfClasses, self.bert.config.hidden_size, (self.bert.config.hidden_size * 2 + 300) , dropout_val)
  def forward(self, input_seq, attention_mask, emoji, hashTag):
    bert_pooled_output = self.bert(input_seq, attention_mask=attention_mask)[0]
    bert_pooled_output = bert_pooled_output[:, 0, :]
    bert_pooled_out_feat = torch.cat([bert_pooled_output, emoji, hashTag], axis = 1)
    # print("Shape",bert_pooled_out_feat.shape)
    output = self.classifier(bert_pooled_out_feat)
    return output

In [4]:
device = xm.xla_device()
model_name = 'adaptive'
model_loc = '/content/drive/My Drive/hasoc_saved/'

In [5]:
def modelEvaluate(model, valid_dataloader, task = 1):
  gc.collect()
  if task == 1:
    taskIndex = 6
  elif task == 2:
    taskIndex = 7
  model.to(device)
  model.eval()
  predictions, true_labels = [], []
  logits = []
  # Predict 
  for batch in valid_dataloader:
    # Add batch to GPU
    b_input_ids = batch[1].to(device)
    b_input_mask = batch[2].to(device)
    b_labels = batch[taskIndex].to(device)
    b_emoji = batch[5].to(device)
    b_hashtag = batch[4].to(device)
    with torch.no_grad():
      pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
    logits.append(pred.detach().cpu().numpy())
    label_ids = b_labels.to('cpu').numpy()
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)
    flat_true_labels = np.concatenate(true_labels, axis = 0)
    predictions = []
    for i in logits:
      for j in i:
        predictions.append(j)
    flat_predictions = [np.argmax(i) for i in predictions]
    assert(len(flat_predictions) == len(flat_true_labels))
    return flat_predictions, flat_true_labels

In [6]:
path =  model_name + ".pt"
scale = 1

In [7]:
def make_optim(model, rate = 2e-5):
  return AdamW(model.parameters(),
                lr = rate, # default = 5e-5, using 2e-5
                eps = 1e-8) # default = 1e-8

def train_model(train_dataloader, valid_dataloader, numberOfEpochs = 10, task = 1):
  """ Train Loop for the model """
  scale = 1
  if task == 2:
    classNum = 4
    taskIndex = 7
  elif task == 1:
    classNum = 2
    taskIndex = 6
  else:
    raise NameError("Task not defined")
  total_steps = len(train_dataloader)
  print("Start")

  model = TextClassification(classNum) # task 1 
  model.to(device)
  # if torch.device == "gpu":
  #   model.cuda()
  # device = torch.device("cuda")
  
  loss_function = nn.CrossEntropyLoss().to(device)
  epoch_loss = 0
  batch_accuracy_scores = []
  global_pred = []
  global_label = []

  present_rate = 2e-5
  old_best = -1
  epoch = 0

  while(1):
    # when the learn rate falls below a lower threshold, you stop your training
    # until that moment, march on
    epoch += 1
    print("\nEpoch:", epoch)
    print("Present Rate: " + str(present_rate))
    optimizer = make_optim(model, present_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
    gc.collect()
    model.to(device)
    model.train()
    epoch_loss = 0
    batch_accuracy_scores = []
    train_data_count = float(len(train_dataloader))

    # to check if performance with default weights
    predictions, true_labels = modelEvaluate(model, valid_dataloader, task)
    score_now = f1_score(true_labels, predictions, average = 'macro')
    print("Validation Macro: " + str(score_now))

    if (score_now > old_best):
      print("Continuing on track")
      old_best = score_now

      # delete previous best 
      delete_filename = path
      open(delete_filename, 'w').close() # overwrite and make the file blank instead
      os.remove(delete_filename) # delete the blank file from google drive will move the file to bin instead
      torch.save({'model':model.state_dict(),
                  'optimizer':optimizer.state_dict(),
                  'f1-score':score_now,
                  'epoch':epoch},'drive/MyDrive/XLM_Data/code_mixed_data_model.pt')
      # torch.save(model.state_dict(), path)

    else:
      print("Backtrack")
      model.load_state_dict(torch.load('drive/MyDrive/XLM_Data/code_mixed_data_model.pt')['model'])
      present_rate /= (4 * scale)
      scale *= 4
      if present_rate < 1e-8:
        break

    # For quick eval
    cnt = 0
    # for i, batch in tqdm(enumerate(train_dataloader)):
    for i, batch in enumerate(train_dataloader):
        print("Iter: " + str(cnt + 1))
        # COMMENT OUT THE NEXT 2 LINES IN ACTUAL TRAINING
        # if cnt == 4:
        #   break
        cnt += 1
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[taskIndex].to(device)
        b_emoji = batch[5].to(device)
        b_hashtag = batch[4].to(device)
        pred = model(b_input_ids,b_input_mask ,b_emoji.float(), b_hashtag.float())
        loss = loss_function(pred.view(-1, classNum), b_labels.view(-1))
        with torch.no_grad():
          epoch_loss += (loss.item() * len(b_labels))
          global_pred.append(pred)
          global_label.append(b_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

  return model

In [8]:
# torch.save({'code_mixed_train_dataloaders':train_dataloader,
#             'code_mixed_valid_dataloaders':valid_dataloader},'drive/MyDrive/XLM_Data/code_mixed_dataloaders.pt')
dataloaders = torch.load("drive/MyDrive/XLM_Data/code_mixed_dataloaders.pt")

In [11]:
gc.collect()
model = train_model(train_dataloader, valid_dataloader, 2, task = 1) 

Start

Epoch: 1
Present Rate: 2e-05
Validation Macro: 0.4482758620689655
Continuing on track
Iter: 1
Iter: 2
Iter: 3
Iter: 4
Iter: 5
Iter: 6
Iter: 7


RuntimeError: ignored